<a href="https://colab.research.google.com/github/MarcusPauloFaustino1/Exercicios-Growdev/blob/main/Trabalho_Final_Prog_DS_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importações

import pandas as pd
import numpy as np
import csv

# leitura de arquivos

pokemon = pd.read_csv('pokemon.csv')

pokemon_tipos_1 = pd.read_csv('pokemon_tipos_1.csv')

pokemon_tipos_2 = pd.read_csv('pokemon_tipos_2.csv')

# informações sobre arquivos

print(pokemon.head(10)) # A coluna "#" domonstrou-se desnecessária 

print(pokemon.info()) # 414 pokemons possuem apenas o tipo primário
                      # Não encontrou-se inconsistências

print("\n\n")

print(pokemon_tipos_1.info())

print("\n\n")

print(pokemon_tipos_2.info()) 

print(pokemon.describe()) # Obsevando o describe, não há outliers.

duplicados = pokemon[pokemon.duplicated(keep='first')] # Não há dados duplicados

duplicados.info()




In [ ]:
# alterações

pokemon.drop(['#'],axis = 1, inplace = True) # excluindo a coluna "#"

pokemon['pokemon_tipo_2_id'] = pokemon['pokemon_tipo_2_id'].fillna(0) # preenchendo NaN para 0

pokemon['pokemon_tipo_2_id'] = pokemon['pokemon_tipo_2_id'].astype('int16') # alterando tipo de dado, de float64 para int16

pokemon_tipos_2.loc[18] = [0,'-'] # adicionando nova legenda para pokemons que não possuem tipo secundário

pokemon = pokemon.astype({col: 'int16' for col in pokemon.select_dtypes('int64').columns}) # convertendo int64 para int16

print(pokemon.info()) # conferindo informações atualizadas

,pokemon_nome,pokemon_tipo_1_id,pokemon_tipo_2_id,pokemon_total,pokemon_hp,pokemon_attack,pokemon_defense,pokemon_ataque_especial,pokemon_defesa_especial,pokemon_velocidade,pokemon_geracao,pokemon_lendario
0,Bulbasaur,10,14,318,45,49,49,65,65,45,1,False
1,Ivysaur,10,14,405,60,62,63,80,80,60,1,False
2,Venusaur,10,14,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,10,14,625,80,100,123,122,120,80,1,False
4,Charmander,7,0,309,39,52,43,60,50,65,1,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   pokemon_nome             800 non-null    object
 1   pokemon_tipo_1_id        800 non-null    int16 
 2   pokemon_tipo_2_id        800 non-null    int16 
 3   pokemon_total            800 non-null    int16 
 4   pokemon_hp               800 non-null    int16 
 5   pokemon_attack           800 non-null    int16 
 6   pokemon_defense          800 non-null    int16 
 7   pokemon_ataque_especial  800 non-null    int16 
 8   pokemon_defesa_especial  800 non-null    int16 
 9   pokemon_velocidade       800 non-null    int16 
 10  pokemon_geracao          800 non-null    int16 
 11  pokemon_lendario         800 non-null    bool  
dtypes: bool(1), int16(10), object(1)
memory usage: 22.8+ KB
None


In [ ]:
pkm_todos_tipos = pokemon.merge(pokemon_tipos_1, how = 'outer').merge(pokemon_tipos_2, how = 'outer')

pkm_todos_tipos.query('pokemon_attack > 100 & pokemon_geracao == 3 & (pokemon_tipo_1_nome == "Grass" | pokemon_tipo_2_nome == "Grass")')

,pokemon_nome,pokemon_tipo_1_id,pokemon_tipo_2_id,pokemon_total,pokemon_hp,pokemon_attack,pokemon_defense,pokemon_ataque_especial,pokemon_defesa_especial,pokemon_velocidade,pokemon_geracao,pokemon_lendario,pokemon_tipo_1_nome,pokemon_tipo_2_nome
550,SceptileMega Sceptile,10,3,630,70,110,75,145,85,145,3,False,Grass,Dragon
570,Cacturne,10,2,475,70,115,60,115,60,55,3,False,Grass,Dark
588,Breloom,10,6,460,60,130,80,60,60,70,3,False,Grass,Fighting


In [95]:

def soma(values):
    soma_valores = values.sum()
    return soma_valores

pokemons_tipo_geracao_1 = pd.pivot_table(pkm_todos_tipos,
    index=['pokemon_tipo_1_nome'],  
    columns=['pokemon_geracao'],
    aggfunc={'pokemon_geracao': soma},
    fill_value =0
)

pokemons_tipo_geracao_2 = pd.pivot_table(pkm_todos_tipos,
    index=['pokemon_tipo_2_nome'],  
    columns=['pokemon_geracao'],
    aggfunc={'pokemon_geracao': soma},
    fill_value =0
)

pokemons_tipo_geracao_1['pokemon_geracao']
pokemons_tipo_geracao_2['pokemon_geracao']

df_total = pokemons_tipo_geracao_1['pokemon_geracao'] + pokemons_tipo_geracao_2['pokemon_geracao'].drop(["-"])

df_total.where(df_total > 50, 0)


pokemon_geracao,1,2,3,4,5,6
pokemon_tipo_1_nome,,,,,,
Bug,0,0,0,0,90,0
Dark,0,0,0,0,80,0
Dragon,0,0,0,0,60,54
Electric,0,0,0,0,60,0
Fairy,0,0,0,0,0,84
Fighting,0,0,0,0,85,0
Fire,0,0,0,0,80,0
Flying,0,0,0,64,105,0
Ghost,0,0,0,0,0,90


In [98]:
import numpy as np

pokemon_velocidade_maior_85 = pkm_todos_tipos.query('pokemon_velocidade > 85')

pokemons_lendarios_velocidade = pd.pivot_table(pokemon_velocidade_maior_85,
              index = ['pokemon_geracao'],
              columns=['pokemon_lendario'],
              aggfunc={'pokemon_geracao': np.size})

display(pokemons_lendarios_velocidade)

pokemon_geracao     
pokemon_lendario           False True
pokemon_geracao                      
1                             49    5
2                             14    4
3                             25   15
4                             32   11
5                             37   15
6                             14    4

In [144]:
'''5) Plote um gráfico mostrando o valor somado da coluna
pokemon_total por tipo de pokemon, contando apenas os
pokemons que possuem defesa menor que 50.'''


def soma(values):
    soma_valores = values.sum()
    return soma_valores

pokemons_defesa_menor_50_tipo_1 = pkm_todos_tipos.query('pokemon_defense < 50 & pokemon_tipo_2_id == 0')

pokemons_total_tipo_defesa_1 = pd.pivot_table(pokemons_defesa_menor_50_tipo_1,
                        index = ['pokemon_tipo_1_nome'],
                        aggfunc={'pokemon_total': soma}
                        )

pokemons_total_tipo_defesa_1 

pokemons_defesa_menor_50_tipo_2 = pkm_todos_tipos.query('pokemon_defense < 50 & pokemon_tipo_2_id != 0')

pokemons_total_tipo_defesa_2 = pd.pivot_table(pokemons_defesa_menor_50_tipo_2,
                        index = ['pokemon_tipo_2_nome'],
                        aggfunc={'pokemon_total': soma}
                        )

#pokemons_total_tipo_defesa_1 = pokemons_total_tipo_defesa_1.rename(columns = {'pokemon_total':'pokemon_total_a'})

pokemons_total_tipo_defesa_3 = pd.concat([pokemons_total_tipo_defesa_1,pokemons_total_tipo_defesa_2],axis = 1)

pokemons_total_tipo_defesa_3 = pokemons_total_tipo_defesa_3.fillna(0)

pokemons_total_tipo_defesa_3 = pokemons_total_tipo_defesa_3.sum(axis = 1)

display(pokemons_total_tipo_defesa_3)


Bug         1818.0
Dark        2908.0
Dragon       845.0
Electric    3391.0
Fairy       3106.0
Fighting    1337.0
Fire        4077.0
Ghost        531.0
Grass       2700.0
Ground      2236.0
Ice          790.0
Normal      6262.0
Poison      4538.0
Psychic     4406.0
Rock         350.0
Water       4176.0
Flying      7024.0
dtype: float64